# EDINETから「主な相手先別の販売実績」を抽出する

import libraries

In [2]:
!pip install edinet_xbrl

In [3]:
import requests
import pandas as pd
import zipfile
import os
from glob import glob
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
from typing import Dict, List
from datetime import datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup

EDINET API より財務情報の一覧を取得

In [19]:
END_POINT = 'https://disclosure.edinet-fsa.go.jp/api/v1'
submission_info_endpoint = f'{END_POINT}/documents.json'
submission_request_parameters = {
    'date': '2022-04-03',
    'type': 2
}
submission_info_response = requests.get(
    submission_info_endpoint, submission_request_parameters
)
submission_info_json = submission_info_response.json()

In [20]:
raw_submission_info_df = pd.DataFrame(submission_info_json['results'])
raw_submission_info_df.columns

Index([], dtype='object')

In [21]:
if any(col not in raw_submission_info_df.columns for col in ['docID', 'edinetCode', 'secCode', 'filerName', 'docDescription']):
    print(1)

1


In [22]:
# 重要なカラムに絞る
submission_info_df = raw_submission_info_df[['docID', 'edinetCode', 'secCode', 'filerName', 'docDescription']]
submission_info_df.head()

KeyError: "None of [Index(['docID', 'edinetCode', 'secCode', 'filerName', 'docDescription'], dtype='object')] are in the [columns]"

In [13]:
# 有価証券報告書の情報を抽出する。
securities_report_infos = []
for i, row in submission_info_df.iterrows():
    doc_desc = row['docDescription']
    doc_seccode = row['secCode']
    
    if doc_desc is None:
        continue
    
    if doc_seccode is None:
        continue
    
    if ('有価証券報告書' in doc_desc) and ('受益証券' not in doc_desc) and ('訂正' not in doc_desc) and ('外国投資証券' not in doc_desc):
        row_to_dataframe = pd.DataFrame([row])
        securities_report_infos.append(row_to_dataframe)

if len(securities_report_infos) == 0:
    print('有価証券報告書の提出情報がありません。')
else:
    print(f'{len(securities_report_infos)} 件の有価証券報告書が抽出されました。')
    securities_report_info_df = pd.concat(securities_report_infos)

有価証券報告書の提出情報がありません。


In [14]:
securities_report_info_df.iloc[70:80]

NameError: name 'securities_report_info_df' is not defined

In [9]:
securities_report_info_df['docID']

188    S100NN6X
210    S100NN81
246    S100NN9G
Name: docID, dtype: object

In [10]:
output_df = pd.DataFrame(columns=['相手先', '前連結_金額（百万円）', '前連結_割合（%）', '当連結_金額（百万円）', '当連結_割合（%）'])

for docID in securities_report_info_df['docID']:
    
    label = securities_report_info_df[securities_report_info_df['docID'] == docID]
    document_endpoint = f'{END_POINT}/documents/{docID}'
    document_request_parameters = {
        'type': 1
    }
    document_response = requests.get(document_endpoint, document_request_parameters)
    
    # まず、返ってきたデータを zip 形式で保存する。
    zip_file_full_path = f'D:/EDINET_DATA/{docID}.zip'
    with open(zip_file_full_path, 'wb') as f:
        for chunk in document_response.iter_content(chunk_size=1024):
            f.write(chunk)
    
    # zip ファイルを解凍する
    os.makedirs(f'D:/EDINET_DATA/{docID}', exist_ok=True)
    with zipfile.ZipFile(zip_file_full_path) as zip_f:
        zip_f.extractall(f'D:/EDINET_DATA/{docID}')
    
    # xbrl ファイルを発見する
    # PublicDoc 内に格納されている xbrl ファイルが分析対象となるファイルである。
    xbrl_expression = f'D:/EDINET_DATA/{docID}/**/PublicDoc/**/*.xbrl'
    xbrl_paths = glob(xbrl_expression, recursive=True)
    
    parser = EdinetXbrlParser()
    # Step2で特定した XBRL ファイルのパスを選択
    xbrl_path = xbrl_paths[0]
    parsed_xbrl = parser.parse_file(xbrl_path)
    
    
    # 経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析 [テキストブロック]の取得
    key = 'jpcrp_cor:ManagementAnalysisOfFinancialPositionOperatingResultsAndCashFlowsTextBlock'
    context_ref = 'FilingDateInstant'
    extracted_data = parsed_xbrl.get_data_by_context_ref(key, context_ref)
    ManagementAnalysis = extracted_data.get_value()
    
    def extract_paragraph_and_following_table(html_code, target_text):
        # BeautifulSoupを使ってパース
        soup = BeautifulSoup(html_code, 'html.parser')

        # 特定の文字列を含むパラグラフを抽出
        target_paragraphs = [p for p in soup.find_all('p') if target_text in p.text]

        # 対応する表を抽出
        tables = []
        for paragraph in target_paragraphs:
            table = paragraph.find_next('table')
            if table:
                tables.append(table)

        return target_paragraphs, tables
    
    
    def extract_table_content(table):
        # テーブルの行を取得
        rows = table.find_all('tr')

        # 各行のデータを取得
        table_data = []
        for row in rows:
            cols = row.find_all(['th', 'td'])
            cols = [col.text.strip() for col in cols]
            table_data.append(cols)

        return table_data

    # 使用例
    html_code = ManagementAnalysis

    # 例として特定の文字列を含むパラグラフとその次に続く表を抽出する関数を呼び出す
    target_text = "主な相手先別"
    paragraphs, tables = extract_paragraph_and_following_table(html_code, target_text)
    
    for p in paragraphs:
        if '主な相手先別' in p.get_text():
            supplier_text = p.get_text()
    
    if '省略' in supplier_text:
        # table_content = [[], [], ['-', '-', '-', '-', '-']]
        table_content = []
    else:
        # 抽出したテーブルの内容を表示
        if tables:
            for table in tables:
                table_content = extract_table_content(table)
        else:
            # table_content = [[], [], ['-', '-', '-', '-', '-']]
            table_content = []
    
    # テーブルの内容をデータフレームに変換
    additional_columns = ['相手先', '前連結_金額（百万円）', '前連結_割合（%）', '当連結_金額（百万円）', '当連結_割合（%）']

    result = [item for item in table_content if len(item) == 5]
    
    # データフレームを結合
    for element in result:
        label[additional_columns] = element
        output_df = pd.concat([label, output_df], axis=0)
    
    # display(output_df)

c:\Users\hisan\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


NameError: name 'supplier_text' is not defined

In [ ]:
output_df.head()

,docID,edinetCode,secCode,filerName,docDescription,相手先,前連結_金額（百万円）,前連結_割合（%）,当連結_金額（百万円）,当連結_割合（%）
1973,S100RB3P,E33615,65620,株式会社ジーニー,有価証券報告書－第13期(2022/04/01－2023/03/31),ヤフー株式会社,"1,429,868",9.9,"507,082",7.9
1973,S100RB3P,E33615,65620,株式会社ジーニー,有価証券報告書－第13期(2022/04/01－2023/03/31),Google Inc.,"5,138,102",35.5,"888,404",13.8
1943,S100RB26,E34499,76770,株式会社ヤシマキザイ,有価証券報告書－第79期(2022/04/01－2023/03/31),西日本旅客鉄道(株),"2,554,404",9.0,"2,353,124",9.2
1943,S100RB26,E34499,76770,株式会社ヤシマキザイ,有価証券報告書－第79期(2022/04/01－2023/03/31),東海旅客鉄道(株),"3,318,227",11.7,"3,525,736",13.8
1943,S100RB26,E34499,76770,株式会社ヤシマキザイ,有価証券報告書－第79期(2022/04/01－2023/03/31),東日本旅客鉄道(株),"4,454,802",15.7,"3,808,360",14.9
